In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
import torch.optim as optim
from torch.autograd import Variable
from data_processor import EyeDataset
from torch.nn.functional import log_softmax

490
4


In [2]:
num_epochs = 10
num_classes = 5
learning_rate = 0.001

### Load the data

In [3]:
eye_dataset = EyeDataset('../classification_data/dataset')
print(len(eye_dataset), 'training examples')

data_loader = torch.utils.data.DataLoader(dataset=eye_dataset, batch_size=10, shuffle=True)

index = 0
for img, label in data_loader:
    print(label.shape, img[0].shape, index)
    index += 1

490 training examples
torch.Size([10]) torch.Size([3, 140, 250]) 0
torch.Size([10]) torch.Size([3, 140, 250]) 1
torch.Size([10]) torch.Size([3, 140, 250]) 2
torch.Size([10]) torch.Size([3, 140, 250]) 3
torch.Size([10]) torch.Size([3, 140, 250]) 4
torch.Size([10]) torch.Size([3, 140, 250]) 5
torch.Size([10]) torch.Size([3, 140, 250]) 6
torch.Size([10]) torch.Size([3, 140, 250]) 7
torch.Size([10]) torch.Size([3, 140, 250]) 8
torch.Size([10]) torch.Size([3, 140, 250]) 9
torch.Size([10]) torch.Size([3, 140, 250]) 10
torch.Size([10]) torch.Size([3, 140, 250]) 11
torch.Size([10]) torch.Size([3, 140, 250]) 12
torch.Size([10]) torch.Size([3, 140, 250]) 13
torch.Size([10]) torch.Size([3, 140, 250]) 14
torch.Size([10]) torch.Size([3, 140, 250]) 15
torch.Size([10]) torch.Size([3, 140, 250]) 16
torch.Size([10]) torch.Size([3, 140, 250]) 17
torch.Size([10]) torch.Size([3, 140, 250]) 18
torch.Size([10]) torch.Size([3, 140, 250]) 19
torch.Size([10]) torch.Size([3, 140, 250]) 20
torch.Size([10]) torch

## Todo: Data Augmentation

### Define the model (add batch norm, dropout, etc later)

In [10]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        NUM_CLASSES = 5
        
        # input shape: [3, 140, 250]
        # conv --> relu
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(5, 5), stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)


        
        
        # input shape: [16, 140, 250]
        # conv --> relu --> max pool
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(5, 5), stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        
        # input shape: [16, 70, 125]
        # conv --> relu --> max pool
#         self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(5,5), stride=1, padding=0)
#         self.relu3 = nn.ReLU()
#         self.pool2 = nn.MaxPool2d(kernel_size=5)
        
        # input shape: [16, 14, 25]
        self.linear = nn.Linear(30208, NUM_CLASSES)
        # minibatch size = 10
#         self.output = nn.Linear(10, 1)
        
    def forward(self, x):

#         print(x.shape)
        x = self.conv1(x)
#         print(x.shape)
        x = self.relu1(x)
#         print(x.shape)
        x = self.pool1(x)
#         print(x.shape)

        
        x = self.conv2(x)
#         print(x.shape)
        x = self.relu2(x)
#         print(x.shape)
        x = self.pool2(x)
#         print(x.shape)
        
        
        # flatten
        x = x.view(x.shape[0], -1)
#         print(x.shape)
        output = self.linear(x)
#         print(output.shape)
        
        return output

In [11]:
model = ConvNet()
print(model)

ConvNet(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=30208, out_features=5, bias=True)
)


### Experiment with different optimizers

In [12]:
LR = 0.001
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

### Train and Test

In [ ]:
# Train the model
total_step = len(data_loader)

num_epochs = 5

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader):
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
